In [ ]:
import pandas as pd
#Dict approach
import glob
import os
import sys
from pathlib import Path
#from typing import Any, Optional
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import h5py
# Imports
from enum import Enum
from pathlib import Path
from typing import Any, Optional
from torch import nn
import pytorch_lightning as pl
import h5py
from tqdm.notebook import tqdm
#from .transforms import KmerShuffle



In [ ]:
#Load genome loader for salk cluster
sys.path.append('/iblm/netapp/home/jjaureguy/genome_loader/genome-loader/src')

sample_path = os.listdir('/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq')

In [ ]:
# Total Dataset not split 
unified_bed_file_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/unified_bed_df.txt',sep='\t')
frag_tn5_h5_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/frag_tn5_h5_df.txt',sep='\t')

#OHE for AGCT
one_hot_enc_genome = h5py.File('/iblm/netapp/data4/jjaureguy/out_dir/genome_onehot.h5','r')

# Train
frag_tn5_train_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/runai_df/train_frag_tn5_h5_df_runai.txt',sep='\t')
train_bed_file_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/runai_df/valid_df_runai.txt',sep='\t')

# Validation
frag_tn5_valid_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/runai_df/valid_frag_tn5_h5_df_runai.txt',sep='\t')
valid_bed_file_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/runai_df/valid_df_runai.txt',sep='\t')
# Test
frag_tn5_test_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/runai_df/test_frag_tn5_h5_df_runai.txt',sep='\t')
test_bed_file_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/runai_df/test_df_runai.txt',sep='\t')




In [ ]:
unified_bed_file_df
df_experimental = unified_bed_file_df.copy(deep=True)

In [ ]:
df_experimental['Contains_n'] = ""
df_experimental['Counter'] = ""

In [ ]:
# Code to ref for apply function
def foo(row):
    print(type(row))
    assert False  # because you only need to see this once
df_experimental.apply(lambda row: foo(row), axis=1)

# Pre compute peak files

In [ ]:
def get_mid_point(start, end):
    #np.random.seed(0)
    mid_point = int((start+end)/2)
    #mid_point = np.random.choice(mid_point, 1, replace=True)
    #print('mid pt selection random', (mid_point))
    #mid_point = mid_point.item()
    return mid_point
    
def get_x_window(mid_point):
    bounds = (mid_point-512, mid_point + 512)
    #self.d['1024_bp_start_end'] = bounds
    bounds = range(mid_point-512, mid_point + 512)
    mid_point = list(bounds)
    return mid_point

len_ds = []

def generate_peak_file(one_hot_enc_genome, df_experimental):
    #for index in range(len(unified_bed_file_df)):
        peak_index = (index // len(df_experimental))
        coords = df_experimental.iloc[peak_index]
        window = get_mid_point(coords.start,coords.end)
        temp = get_x_window(mid_point=window)
        x = one_hot_enc_genome[coords.chrom]['onehot'][temp]
        #Checks if [0000] OHE vector which represents N's are in the ndarray and returns None if they are
        if ([0,0,0,0] == x).all(axis = 1).any():
            #print('true')
            #f_read_depth_h5.loc[df_read_depth_h5['imbalanced_treatments'] == False]
            #df.loc[df.index[0], 'a']
            #df_experimental.loc[peak_index, 'Contains_n'] = True
            len_ds.append('True')
            
        
        else:
            #df_experimental.loc[peak_index, 'Contains_n'] = False             
            len_ds.append('False')
def foo(row):
    counter = 0
    #print('row',row)
    window = get_mid_point(row.start,row.end)
    temp = get_x_window(mid_point=window)
    x = one_hot_enc_genome[row.chrom]['onehot'][temp]
    #row.Contains_n = True
    if ([0,0,0,0] == x).all(axis = 1).any():
        df_experimental['Contains_n'] = True
        #print('true')
        #print(x)
        counter +=1
        df_experimental['Counter'] = counter
    else:
        df_experimental['Contains_n'] = False
    return counter

In [ ]:
df_experimental['Contains_n'] = df_experimental.apply(lambda row: foo(row), axis=1)

In [ ]:
df_experimental['Counter'] = df_experimental.apply(lambda row: foo(row), axis=1)

In [ ]:
df_experimental.value_counts('Counter')

In [ ]:
df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/runai_df/df_filtered.txt', sep='\t')

In [ ]:
df

In [ ]:
df.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
df

In [ ]:
 filtered_df = df.loc[df['Contains_n'] == 0]

In [ ]:
filtered_df = filtered_df.reset_index(drop=True)

In [ ]:
train_df = filtered_df.loc[filtered_df['train'] == True, ['chrom', 'start', 'end']]
valid_df = filtered_df.loc[filtered_df['valid'] == True, ['chrom', 'start', 'end']]
test_df = filtered_df.loc[filtered_df['test'] == True, ['chrom', 'start', 'end']]

In [ ]:
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
train_df.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/runai_df/train_peak_df_filtered.txt', sep='\t', header=True)
valid_df.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/runai_df/valid_peak_df_filtered.txt', sep='\t', header=True)
test_df.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/runai_df/test_peak_df_filtered.txt', sep='\t', header=True)